# Traces

Effectively manage traces in your agent workflow

## 1. Automatic Trace Management

The simplest way to create and manage traces is to use the init function with automatic trace creation:

```python
import agentops

# Initialize with automatic trace creation (default)
agentops.init(api_key="YOUR_API_KEY", default_tags=["production"])
```

## [2. Manual Trace Creation](https://docs.agentops.ai/v2/usage/manual-trace-control)

For more control, you can disable automatic trace creation and start traces manually

### Basic Manual Trace Control

#### Starting and Ending Traces

The most basic form of manual trace control involves starting a trace, executing your code, and then ending the trace with a specific state:

```python
import agentops

# Initialize without automatic session creation
agentops.init("your-api-key", auto_start_session=False)

# Start a trace manually
trace = agentops.start_trace("my-workflow")

try:
    # Your application logic here
    result = perform_some_operation()
    
    # End the trace successfully
    agentops.end_trace(trace, "Success")
except Exception as e:
    # End the trace with failure state
    agentops.end_trace(trace, "Indeterminate")
```

#### Trace Names and Tags

Trace Names and Tags

```python
# Start a trace with custom name and tags
trace = agentops.start_trace(
    trace_name="customer-service-workflow",
    tags=["customer-123", "priority-high", "support"]
)
```

#### Batch Processing with Selective Trace Ending

For batch processing scenarios, you can selectively end traces based on processing results:

```python
import agentops

# Initialize AgentOps
agentops.init("your-api-key", auto_start_session=False)

# Sample batch items to process
batch_items = [
    {"id": 1, "data": "item_1_data", "valid": True},
    {"id": 2, "data": "item_2_data", "valid": False},
    {"id": 3, "data": "item_3_data", "valid": True},
]
@agentops.operation(name="process_item")
def process_item(item):
    """Simulate processing an item"""
    if not item.get("valid", False):
        raise ValueError(f"Invalid item: {item['id']}")
    return {"processed": True, "result": f"Processed {item['data']}"}

# Start traces for batch items
for i, item in enumerate(batch_items):
    trace = agentops.start_trace(f"batch_item_{i+1}")
    try:
        result = process_item(item)
        if result.get("processed"):
            agentops.end_trace(trace, "Success")
        else:
            agentops.end_trace(trace, "Indeterminate")
    except Exception as e:
        agentops.end_trace(trace, "Error")
```

### Updating Trace Metadata During Execution

You can update metadata on running traces at any point during execution using the `update_trace_metadata` function. This is useful for adding context, tracking progress, or storing intermediate results.

#### Basic Metadata Updates

```python
import agentops

# Initialize AgentOps
agentops.init("your-api-key", auto_start_session=False)

# Start a trace with initial tags
trace = agentops.start_trace("ai-agent-workflow", tags=["startup", "initialization"])

# Your AI agent code runs here...
process_user_request()

# Update metadata with results
agentops.update_trace_metadata({
    "operation_name": "AI Agent Processing Complete", 
    "stage": "completed",
    "response_quality": "high",
    "tags": ["ai-agent", "completed", "success"]  # Tags show current status
})

# End the trace
agentops.end_trace(trace, "Success")
```

#### Semantic Convention Support

The function automatically maps user-friendly keys to semantic conventions when possible:

```python
# These keys will be mapped to semantic conventions
agentops.update_trace_metadata({
    "operation_name": "AI Agent Data Processing",
    "tags": ["production", "batch-job", "gpt-4"],  # Maps to core.tags
    "agent_name": "DataProcessorAgent",             # Maps to agent.name
    "workflow_name": "Intelligent ETL Pipeline",   # Maps to workflow.name
})
```

#### Advanced Metadata with Custom Prefix

You can specify a custom prefix for your metadata attributes:

```python
# Use a custom prefix for business-specific metadata
agentops.update_trace_metadata({
    "customer_id": "CUST_456",
    "order_value": 99.99,
    "payment_method": "credit_card",
    "agent_interaction": "customer_support"
}, prefix="business")

# Results in:
# business.customer_id = "CUST_456"
# business.order_value = 99.99
# business.payment_method = "credit_card"
```

#### Real-World Example: Progress Tracking

Here’s how to use metadata updates to track progress through a complex workflow:

```python
import agentops
from agentops.sdk.decorators import operation

agentops.init(auto_start_session=False)

@operation
def process_batch(batch_data):
    # Simulate batch processing
    return f"Processed {len(batch_data)} items"

def run_etl_pipeline(data_batches):
    """ETL pipeline with progress tracking via metadata"""
    
    trace = agentops.start_trace("etl-pipeline", tags=["data-processing"])
    
    total_batches = len(data_batches)
    processed_records = 0
    
    # Initial metadata
    agentops.update_trace_metadata({
        "operation_name": "ETL Pipeline Execution",
        "pipeline_stage": "starting",
        "total_batches": total_batches,
        "processed_batches": 0,
        "processed_records": 0,
        "estimated_completion": "calculating...",
        "tags": ["etl", "data-processing", "async-operation"]
    })
    
    try:
        for i, batch in enumerate(data_batches):
            # Update progress
            agentops.update_trace_metadata({
                "pipeline_stage": "processing",
                "current_batch": i + 1,
                "processed_batches": i,
                "progress_percentage": round((i / total_batches) * 100, 2)
            })
            
            # Process the batch
            result = process_batch(batch)
            processed_records += len(batch)
            
            # Update running totals
            agentops.update_trace_metadata({
                "processed_records": processed_records,
                "last_batch_result": result
            })
        
        # Final metadata update
        agentops.update_trace_metadata({
            "operation_name": "ETL Pipeline Completed",
            "pipeline_stage": "completed",
            "processed_batches": total_batches,
            "progress_percentage": 100.0,
            "completion_status": "success",
            "total_execution_time": "calculated_automatically",
            "tags": ["etl", "completed", "success"]
        })
        
        agentops.end_trace(trace, "Success")
        
    except Exception as e:
        # Error metadata
        agentops.update_trace_metadata({
            "operation_name": "ETL Pipeline Failed",
            "pipeline_stage": "failed",
            "error_message": str(e),
            "completion_status": "error",
            "failed_at_batch": i + 1 if 'i' in locals() else 0,
            "tags": ["etl", "failed", "error"]
        })
        
        agentops.end_trace(trace, "Error")
        raise

# Example usage
data_batches = [
    ["record1", "record2", "record3"],
    ["record4", "record5"],
    ["record6", "record7", "record8", "record9"]
]

run_etl_pipeline(data_batches)
```

#### Supported Data Types

The `update_trace_metadata` function supports various data types:

```python
agentops.update_trace_metadata({
    "operation_name": "Multi-type Data Example",
    "successful_operation": True,
    "tags": ["example", "demo", "multi-agent"],
    "processing_steps": ["validation", "transformation", "output"]
})

# Note: Lists are automatically converted to JSON strings for OpenTelemetry compatibility
```

## Integration with Decorators

Manual trace control works seamlessly with AgentOps decorators:

```python
import agentops
from agentops.sdk.decorators import agent, operation, tool

agentops.init("your-api-key", auto_start_session=False)

@agent
class CustomerServiceAgent:
    @operation
    def analyze_request(self, request):
        return f"Analyzed: {request}"
    
    @tool(cost=0.02)
    def lookup_customer(self, customer_id):
        return f"Customer data for {customer_id}"

# Manual trace with decorated components
trace = agentops.start_trace("customer-service")

try:
    agent = CustomerServiceAgent()
    customer_data = agent.lookup_customer("CUST_123")
    analysis = agent.analyze_request("billing issue")
    
    agentops.end_trace(trace, "Success")
except Exception as e:
    agentops.end_trace(trace, "Error")
```

## Real-World Example

Here’s a comprehensive example showing manual trace control in a customer service application:

```python
import agentops
from agentops.sdk.decorators import agent, operation, tool
from openai import OpenAI

agentops.init(auto_start_session=False)
client = OpenAI()

@operation
def analyze_sentiment(text):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": f"Analyze sentiment: {text}"}]
        )
        return response.choices[0].message.content.strip()
    
@tool(cost=0.01)
def lookup_order(order_id):
    return f"Order {order_id} details"

def process_customer_requests(requests):
    """Process multiple customer requests with individual trace tracking"""
    results = []
    for i, request in enumerate(requests):
        trace = agentops.start_trace(
            f"customer_request_{i+1}",
            tags=["customer-service", request.get("priority", "normal")]
        )
        try:
            sentiment = analyze_sentiment(request["message"])
            
            if "order" in request:
                order_info = lookup_order(request["order"])
            
            if "positive" in sentiment.lower() or "neutral" in sentiment.lower():
                agentops.end_trace(trace, "Success")
                results.append({"status": "resolved", "sentiment": sentiment})
            else:
                agentops.end_trace(trace, "Escalation_Required")
                results.append({"status": "escalated", "sentiment": sentiment})
                
        except Exception as e:
            agentops.end_trace(trace, "Error")
            results.append({"status": "error", "error": str(e)})
    
    return results

customer_requests = [
    {"message": "I love this product!", "priority": "low"},
    {"message": "My order is completely wrong!", "order": "12345", "priority": "high"},
    {"message": "When will my package arrive?", "order": "67890", "priority": "normal"}
]

results = process_customer_requests(customer_requests)
print(f"Processed {len(results)} customer requests")
```

This example demonstrates:

- Individual trace management for each customer request
- Integration with decorated agents and tools
- Different end states based on business logic
- Proper error handling with appropriate trace states
- Use of tags for categorization

## [3. Using the Trace Decorator](https://docs.agentops.ai/v2/usage/trace-decorator)

You can use the `@trace` decorator to create a trace for a specific function.

## Basic Usage

### Simple Trace Creation

The `@trace` decorator automatically creates a trace span that encompasses the entire function execution. You can optionally specify custom names and tags to better organize and categorize your traces:

```python
from agentops.sdk.decorators import trace
import agentops

# Initialize AgentOps
agentops.init("your-api-key", auto_start_session=False)

@trace(name="customer-workflow", tags=["production", "customer-service"])
def my_workflow():
    """A simple workflow wrapped in a trace"""
    print("🚀 Starting customer workflow...")
    print("📋 Processing customer request...")
    # Your application logic here
    print("✅ Customer workflow completed successfully!")
    return "Workflow completed"

# Run the function - this creates and manages the trace automatically
print("🎬 Running traced workflow...")
result = my_workflow()
print(f"📊 Result: {result}")
```

Both `name` and `tags` parameters are optional. If no name is provided, the function name will be used as the trace name.

### Custom Trace Names

You can specify custom names for your traces:

```python
@trace(name="customer-onboarding-flow")
def onboard_customer(customer_data):
    """Customer onboarding process"""
    print(f"👋 Onboarding customer: {customer_data['name']}")
    print("📝 Creating customer profile...")
    print("📧 Sending welcome email...")
    print("✅ Customer onboarding complete!")
    return f"Onboarded customer: {customer_data['name']}"

@trace(name="data-processing-pipeline")
def process_data(input_data):
    """Data processing workflow"""
    print(f"📊 Processing {len(input_data)} data items...")
    print("🔄 Applying transformations...")
    print("✅ Data processing complete!")
    return f"Processed {len(input_data)} items"

# Usage examples
customer = {"name": "Alice Johnson", "email": "alice@example.com"}
result1 = onboard_customer(customer)
print(f"📋 Onboarding result: {result1}")

data_items = ["item1", "item2", "item3", "item4", "item5"]
result2 = process_data(data_items)
print(f"📋 Processing result: {result2}")
```

### Adding Tags to Traces

Tags help categorize and filter traces in your dashboard:

```python
@trace(tags=["production", "high-priority"])
def critical_workflow():
    """Critical production workflow"""
    print("🚨 Executing critical production workflow...")
    print("⚡ High priority processing...")
    print("✅ Critical task completed successfully!")
    return "Critical task completed"

@trace(name="user-analysis", tags=["analytics", "user-behavior"])
def analyze_user_behavior(user_id):
    """Analyze user behavior patterns"""
    print(f"🔍 Analyzing behavior for user: {user_id}")
    print("📈 Gathering user interaction data...")
    print("🧠 Running behavior analysis algorithms...")
    print("✅ User behavior analysis complete!")
    return f"Analysis complete for user {user_id}"

# Usage examples
print("🎬 Running critical workflow...")
result1 = critical_workflow()
print(f"📊 Critical workflow result: {result1}")

print("\n🎬 Running user analysis...")
result2 = analyze_user_behavior("user_12345")
print(f"📊 Analysis result: {result2}")
```

## Integration with Other Decorators

### Combining with Agent and Operation Decorators

The `@trace` decorator works seamlessly with other AgentOps decorators:

```python
import agentops
from agentops.sdk.decorators import trace, agent, operation, tool

# Initialize AgentOps without auto-starting session since we use @trace
agentops.init("your-api-key", auto_start_session=False)

@agent
class DataAnalysisAgent:
    def __init__(self):
        print("🤖 DataAnalysisAgent initialized")
    
    @operation
    def collect_data(self, source):
        print(f"📊 Collecting data from {source}...")
        data = f"Data collected from {source}"
        print(f"✅ Data collection complete: {data}")
        return data
    
    @tool(cost=0.05)
    def analyze_data(self, data):
        print(f"🧠 Analyzing data: {data}")
        analysis = f"Analysis of {data}"
        print(f"✅ Analysis complete: {analysis}")
        return analysis
    
    @operation
    def generate_report(self, analysis):
        print(f"📝 Generating report from: {analysis}")
        report = f"Report: {analysis}"
        print(f"✅ Report generated: {report}")
        return report

@trace(name="complete-analysis-workflow")
def run_analysis_workflow(data_source):
    """Complete data analysis workflow"""
    print(f"🚀 Starting analysis workflow for: {data_source}")
    print("=" * 50)
    
    agent = DataAnalysisAgent()
    
    # Collect data
    print("\n📋 Step 1: Data Collection")
    data = agent.collect_data(data_source)
    
    # Analyze data
    print("\n📋 Step 2: Data Analysis")
    analysis = agent.analyze_data(data)
    
    # Generate report
    print("\n📋 Step 3: Report Generation")
    report = agent.generate_report(analysis)
    
    print("\n🎉 Workflow completed successfully!")
    print("=" * 50)
    
    return {
        "source": data_source,
        "report": report
    }

# Usage
print("🎬 Running complete analysis workflow...")
result = run_analysis_workflow("customer_database")
print(f"\n📊 Final Result:")
print(f"   Source: {result['source']}")
print(f"   Report: {result['report']}")
```

## Async Function Support

The `@trace` decorator fully supports async functions:

```python
import asyncio
import agentops
from agentops.sdk.decorators import trace, operation

# Initialize AgentOps without auto-starting session since we use @trace
agentops.init("your-api-key", auto_start_session=False)

@operation
async def fetch_user_data(user_id):
    """Simulate async data fetching"""
    print(f"🌐 Fetching data for user: {user_id}")
    await asyncio.sleep(1)  # Simulate API call
    data = f"User data for {user_id}"
    print(f"✅ Data fetched: {data}")
    return data

@operation
async def process_user_data(user_data):
    """Simulate async data processing"""
    print(f"⚙️ Processing user data: {user_data}")
    await asyncio.sleep(0.5)  # Simulate processing
    processed = f"Processed: {user_data}"
    print(f"✅ Processing complete: {processed}")
    return processed

@trace(name="async-user-workflow")
async def async_user_workflow(user_id):
    """Async workflow for user processing"""
    print(f"🚀 Starting async workflow for user: {user_id}")
    print("=" * 45)
    
    print("\n📋 Step 1: Fetching user data")
    user_data = await fetch_user_data(user_id)
    
    print("\n📋 Step 2: Processing user data")
    processed_data = await process_user_data(user_data)
    
    print("\n🎉 Async workflow completed!")
    print("=" * 45)
    
    return processed_data

# Usage
async def main():
    print("🎬 Running async user workflow...")
    result = await async_user_workflow("user_123")
    print(f"\n📊 Final Result: {result}")
    print("✨ Check your AgentOps dashboard to see the traced async workflow!")

# Run the async workflow
print("🔄 Starting async demo...")
asyncio.run(main())
```

## Error Handling and Trace States

### Automatic Error Handling

The `@trace` decorator automatically handles exceptions and sets appropriate trace states:

```python
import agentops
from agentops.sdk.decorators import trace

# Initialize AgentOps without auto-starting session since we use @trace
agentops.init("your-api-key", auto_start_session=False)

@trace(name="error-prone-workflow")
def risky_operation():
    """Operation that might fail"""
    import random
    
    print("🎲 Running risky operation...")
    print("⚠️ This operation has a 50% chance of failure")
    
    if random.random() < 0.5:
        print("❌ Operation failed!")
        raise ValueError("Random failure occurred")
    
    print("✅ Operation succeeded!")
    return "Operation succeeded"

# The trace will automatically be marked with failure state if an exception occurs
print("🎬 Testing automatic error handling...")
for i in range(3):
    print(f"\n🔄 Attempt {i+1}:")
    try:
        result = risky_operation()
        print(f"📊 Success: {result}")
        break
    except ValueError as e:
        print(f"📊 Operation failed: {e}")
        print("🔍 Trace automatically ended with error state")
```

### Custom Error Handling

You can implement custom error handling within traced functions:

```python
@trace(name="robust-workflow")
def robust_operation(data):
    """Operation with custom error handling"""
    print(f"🚀 Starting robust operation with data: {data}")
    
    try:
        # Risky operation
        if not data:
            print("⚠️ No data provided!")
            raise ValueError("No data provided")
        
        # Process data
        print("⚙️ Processing data...")
        result = f"Processed: {data}"
        print(f"✅ Processing successful: {result}")
        return {"success": True, "result": result}
        
    except ValueError as e:
        # Handle specific errors
        print(f"❌ Validation error: {e}")
        return {"success": False, "error": str(e)}
    except Exception as e:
        # Handle unexpected errors
        print(f"💥 Unexpected error: {e}")
        return {"success": False, "error": f"Unexpected error: {str(e)}"}

# Usage examples
print("\n🎬 Testing custom error handling...")

print("\n📋 Test 1: Valid data")
result1 = robust_operation("valid_data")
print(f"📊 Result: {result1}")

print("\n📋 Test 2: Empty data")
result2 = robust_operation("")
print(f"📊 Result: {result2}")

print("\n📋 Test 3: None data")
result3 = robust_operation(None)
print(f"📊 Result: {result3}")
```

## Real-World Examples

### E-commerce Order Processing

```python
from agentops.sdk.decorators import trace, agent, operation, tool
from openai import OpenAI
import agentops

agentops.init("your-api-key", auto_start_session=False)

@agent
class OrderProcessor:
    def __init__(self):
        print("🛒 OrderProcessor initialized")
    
    @tool(cost=0.01)
    def validate_payment(self, payment_info):
        """Payment validation service"""
        print(f"💳 Validating payment: {payment_info['card']}")
        result = {"valid": True, "transaction_id": "txn_123"}
        print(f"✅ Payment validation successful: {result['transaction_id']}")
        return result
    
    @tool(cost=0.02)
    def check_inventory(self, product_id, quantity):
        """Inventory check service"""
        print(f"📦 Checking inventory for {product_id} (qty: {quantity})")
        result = {"available": True, "reserved": quantity}
        print(f"✅ Inventory check complete: {quantity} units available")
        return result
    
    @operation
    def calculate_shipping(self, address, items):
        """Calculate shipping costs"""
        print(f"🚚 Calculating shipping to {address['city']}, {address['state']}")
        result = {"cost": 9.99, "method": "standard"}
        print(f"✅ Shipping calculated: ${result['cost']} ({result['method']})")
        return result
    
    @tool(cost=0.005)
    def send_confirmation_email(self, email, order_details):
        """Email service"""
        print(f"📧 Sending confirmation email to {email}")
        result = f"Confirmation sent to {email}"
        print(f"✅ Email sent successfully")
        return result

@trace(name="order-processing", tags=["ecommerce", "orders"])
def process_order(order_data):
    """Complete order processing workflow"""
    print(f"🚀 Starting order processing for {order_data['customer_email']}")
    print("=" * 60)
    
    processor = OrderProcessor()
    
    try:
        # Validate payment
        print("\n📋 Step 1: Payment Validation")
        payment_result = processor.validate_payment(order_data["payment"])
        if not payment_result["valid"]:
            print("❌ Payment validation failed!")
            return {"success": False, "error": "Payment validation failed"}
        
        # Check inventory for all items
        print("\n📋 Step 2: Inventory Check")
        for item in order_data["items"]:
            inventory_result = processor.check_inventory(
                item["product_id"], 
                item["quantity"]
            )
            if not inventory_result["available"]:
                print(f"❌ Item {item['product_id']} not available!")
                return {"success": False, "error": f"Item {item['product_id']} not available"}
        
        # Calculate shipping
        print("\n📋 Step 3: Shipping Calculation")
        shipping = processor.calculate_shipping(
            order_data["shipping_address"], 
            order_data["items"]
        )
        
        # Send confirmation
        print("\n📋 Step 4: Confirmation Email")
        confirmation = processor.send_confirmation_email(
            order_data["customer_email"],
            {
                "items": order_data["items"],
                "shipping": shipping,
                "payment": payment_result
            }
        )
        
        print("\n🎉 Order processing completed successfully!")
        print("=" * 60)
        
        return {
            "success": True,
            "order_id": "ORD_12345",
            "payment": payment_result,
            "shipping": shipping,
            "confirmation": confirmation
        }
        
    except Exception as e:
        print(f"💥 Order processing failed: {e}")
        return {"success": False, "error": str(e)}

# Usage
print("🎬 Running e-commerce order processing demo...")

order = {
    "customer_email": "customer@example.com",
    "payment": {"card": "****1234", "amount": 99.99},
    "items": [{"product_id": "PROD_001", "quantity": 2}],
    "shipping_address": {"city": "New York", "state": "NY"}
}

result = process_order(order)

print(f"\n📊 ORDER PROCESSING RESULT:")
print(f"   Success: {result['success']}")
if result['success']:
    print(f"   Order ID: {result['order_id']}")
    print(f"   Transaction: {result['payment']['transaction_id']}")
    print(f"   Shipping: ${result['shipping']['cost']}")
else:
    print(f"   Error: {result['error']}")
```

### Data Analysis Workflow

```python
from agentops.sdk.decorators import trace, agent, operation, tool
from openai import OpenAI
import agentops

agentops.init("your-api-key", auto_start_session=False)

@agent
class DataAnalysisAgent:
    def __init__(self):
        self.client = OpenAI()
        print("🤖 DataAnalysisAgent initialized")
    
    @operation
    def collect_data(self, source):
        """Simulate data collection"""
        print(f"📊 Collecting data from {source}...")
        data = f"Raw data collected from {source}: [sample_data_1, sample_data_2, sample_data_3]"
        print(f"✅ Data collection complete: {len(data)} characters collected")
        return data
    
    @operation
    def analyze_data_with_llm(self, data):
        """Use LLM to analyze the collected data"""
        print("🧠 Analyzing data with LLM...")
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a data analyst. Analyze the provided data and give insights."},
                {"role": "user", "content": f"Please analyze this data: {data}"}
            ]
        )
        analysis = response.choices[0].message.content
        print(f"✅ LLM analysis complete: {len(analysis)} characters generated")
        return analysis
    
    @tool(cost=0.05)
    def generate_visualization(self, analysis):
        """Generate data visualization"""
        print("📈 Generating visualization...")
        visualization = f"Chart generated for: {analysis[:50]}..."
        print(f"✅ Visualization generated: {visualization}")
        return visualization
    
    @operation
    def generate_report(self, analysis, visualization):
        """Generate final report using LLM"""
        print("📝 Generating final report with LLM...")
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a report writer. Create a professional data analysis report."},
                {"role": "user", "content": f"Create a report based on this analysis: {analysis} and visualization: {visualization}"}
            ]
        )
        report = response.choices[0].message.content
        print(f"✅ Final report generated: {len(report)} characters")
        return report

@trace(name="data-analysis-workflow", tags=["analytics", "reporting"])
def run_data_analysis(data_source):
    """Complete data analysis workflow with LLM integration"""
    print(f"🚀 Starting data analysis workflow for: {data_source}")
    print("=" * 60)
    
    agent = DataAnalysisAgent()
    
    # Collect data
    print("\n📋 Step 1: Data Collection")
    raw_data = agent.collect_data(data_source)
    
    # Analyze data using LLM
    print("\n📋 Step 2: LLM Analysis")
    analysis = agent.analyze_data_with_llm(raw_data)
    
    # Generate visualization
    print("\n📋 Step 3: Visualization Generation")
    visualization = agent.generate_visualization(analysis)
    
    # Generate final report using LLM
    print("\n📋 Step 4: Report Generation")
    report = agent.generate_report(analysis, visualization)
    
    print("\n🎉 Workflow completed successfully!")
    print("=" * 60)
    
    return {
        "source": data_source,
        "raw_data": raw_data,
        "analysis": analysis,
        "visualization": visualization,
        "final_report": report
    }

# Usage
print("🎬 Running data analysis workflow demo...")

result = run_data_analysis("customer_database")

print(f"\n📊 ANALYSIS RESULTS:")
print(f"   Data Source: {result['source']}")
print(f"   Raw Data: {result['raw_data'][:80]}...")
print(f"   Analysis Preview: {result['analysis'][:100]}...")
print(f"   Visualization: {result['visualization']}")
print(f"   Final Report Preview: {result['final_report'][:150]}...")

print(f"\n✨ Analysis complete! Check your AgentOps dashboard to see the traced workflow.")
```

## Best Practices

### 1. Use Meaningful Names
Choose descriptive names that clearly indicate what the trace represents:

```python
# Good
@trace(name="user-authentication-flow")
def authenticate_user(credentials):
    pass

@trace(name="payment-processing-pipeline")
def process_payment(payment_data):
    pass

# Less descriptive
@trace(name="trace1")
def some_function():
    pass
```

### 2. Add Relevant Tags

Use tags to categorize traces for easier filtering and analysis:

```python
@trace(name="order-fulfillment", tags=["ecommerce", "fulfillment", "high-priority"])
def fulfill_order(order_id):
    pass

@trace(name="data-sync", tags=["background-job", "data-processing"])
def sync_data():
    pass
```

### 3. Keep Traces Focused

Each trace should represent a logical unit of work:

```python
# Good - focused on a single workflow
@trace(name="customer-onboarding")
def onboard_customer(customer_data):
    validate_customer(customer_data)
    create_account(customer_data)
    send_welcome_email(customer_data)

# Less focused - mixing different concerns
@trace(name="mixed-operations")
def do_everything():
    onboard_customer(data1)
    process_orders(data2)
    generate_reports(data3)
```

### 4. Handle Errors Appropriately

Implement proper error handling within traced functions:

```python
@trace(name="data-processing")
def process_data(data):
    try:
        # Main processing logic
        result = complex_processing(data)
        return {"success": True, "result": result}
    except ValidationError as e:
        # Expected errors
        return {"success": False, "error": "validation_failed", "details": str(e)}
    except Exception as e:
        # Unexpected errors
        logger.error(f"Unexpected error in data processing: {e}")
        return {"success": False, "error": "processing_failed"}
```

## [4. Trace Context Manager](https://docs.agentops.ai/v2/usage/context-managers)
TraceContext objects support Python’s context manager protocol, making it easy to manage trace lifecycles.

AgentOps provides native context manager support for traces, allowing you to use Python’s `with` statement for automatic trace lifecycle management. This approach ensures traces are properly started and ended, even when exceptions occur.

### Basic Usage

The simplest way to use context managers is with the start_trace() function:

```python
import agentops

# Initialize AgentOps
agentops.init(api_key="your-api-key")

# Use context manager for automatic trace management
with agentops.start_trace("my_workflow") as trace:
    # Your code here
    print("Processing data...")
    # Trace automatically ends when exiting the with block
```

The trace will automatically:

- Start when entering the with block
- End with “Success” status when exiting normally
- End with “Error” status if an exception occurs
- Clean up resources properly in all cases

### Advanced Usage

#### Traces with Tags

You can add tags to traces for better organization and filtering:

```python
import agentops

agentops.init(api_key="your-api-key")

# Using list tags
with agentops.start_trace("data_processing", tags=["batch", "production"]):
    process_batch_data()

# Using dictionary tags for more structured metadata
with agentops.start_trace("user_request", tags={
    "user_id": "12345",
    "request_type": "query",
    "priority": "high"
}):
    handle_user_request()
```

#### Parallel Traces

Context managers create independent parallel traces, not parent-child relationships:

```python
import agentops

agentops.init(api_key="your-api-key")

# Sequential parallel traces
with agentops.start_trace("task_1"):
    print("Task 1 executing")

with agentops.start_trace("task_2"):
    print("Task 2 executing")

# Nested context managers create parallel traces
with agentops.start_trace("outer_workflow"):
    print("Outer workflow started")
    
    with agentops.start_trace("inner_task"):
        print("Inner task executing (parallel to outer)")
    
    print("Outer workflow continuing")
```

#### Exception Handling

Context managers automatically handle exceptions and set appropriate trace states:

```python
import agentops

agentops.init(api_key="your-api-key")

# Automatic error handling
try:
    with agentops.start_trace("risky_operation"):
        # This will automatically set trace status to "Error"
        raise ValueError("Something went wrong")
except ValueError as e:
    print(f"Caught error: {e}")
    # Trace has already been ended with Error status

# Graceful degradation pattern
try:
    with agentops.start_trace("primary_service"):
        result = call_primary_service()
except ServiceUnavailableError:
    with agentops.start_trace("fallback_service"):
        result = call_fallback_service()
```

#### Concurrent Execution

Context managers work seamlessly with threading and asyncio:

```python
import agentops
import threading

agentops.init(api_key="your-api-key")

# With threading
def worker_function(worker_id):
    with agentops.start_trace(f"worker_{worker_id}"):
        # Each thread gets its own independent trace
        process_work(worker_id)

threads = []
for i in range(3):
    thread = threading.Thread(target=worker_function, args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()
```

### Production Patterns

#### API Endpoint Monitoring

```python
import agentops
from flask import Flask, request

app = Flask(__name__)
agentops.init(api_key="your-api-key")

@app.route('/api/process', methods=['POST'])
def process_request():
    # Create trace for each API request
    with agentops.start_trace("api_request", tags={
        "endpoint": "/api/process",
        "method": "POST",
        "user_id": request.headers.get("user-id")
    }):
        try:
            data = request.get_json()
            result = process_data(data)
            return {"status": "success", "result": result}
        except Exception as e:
            # Exception automatically sets trace to Error status
            return {"status": "error", "message": str(e)}, 500
```

#### Batch Processing

```python
import agentops

agentops.init(api_key="your-api-key")

def process_batch(items):
    with agentops.start_trace("batch_processing", tags={
        "batch_size": len(items),
        "batch_type": "data_processing"
    }):
        successful = 0
        failed = 0
        
        for item in items:
            try:
                with agentops.start_trace("item_processing", tags={
                    "item_id": item.get("id"),
                    "item_type": item.get("type")
                }):
                    process_item(item)
                    successful += 1
            except Exception as e:
                failed += 1
                print(f"Failed to process item {item.get('id')}: {e}")
        
        print(f"Batch completed: {successful} successful, {failed} failed")
```

#### Retry Logic

```python
import agentops
import time

agentops.init(api_key="your-api-key")

def retry_operation(operation_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            with agentops.start_trace(f"{operation_name}_attempt_{attempt + 1}", tags={
                "operation": operation_name,
                "attempt": attempt + 1,
                "max_retries": max_retries
            }):
                # Your operation here
                result = perform_operation()
                return result  # Success - exit retry loop
                
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"Attempt {attempt + 1} failed: {e}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"All {max_retries} attempts failed")
                raise
```

### Backward Compatibility

Context managers are fully backward compatible with existing AgentOps code patterns:

- Manual Management

```python
import agentops

agentops.init(api_key="your-api-key")

# Manual trace management (legacy)
trace = agentops.start_trace("manual_trace")
# ... your code ...
agentops.end_trace(trace, "Success")
```

- Context Manager

```python
import agentops

agentops.init(api_key="your-api-key")

# Context manager (new, recommended)
with agentops.start_trace("context_managed_trace") as trace:
    # ... your code ...
    pass  # Automatically ended
```

- Property Access

```python
import agentops

agentops.init(api_key="your-api-key")

# Accessing trace properties
with agentops.start_trace("property_access") as trace:
    span = trace.span  # Access underlying span
    trace_id = trace.span.get_span_context().trace_id
```

- Mixed Usage

```python
import agentops

agentops.init(api_key="your-api-key")

# Accessing trace properties
with agentops.start_trace("property_access") as trace:
    span = trace.span  # Access underlying span
    trace_id = trace.span.get_span_context().trace_id
```